# People counter model preparation and compilation
# Section 1: Model preparation

## Install and import necessary Python libraries
In this first step, we install some useful Python libraries for us to easily prepare the model to be compiled by SageMaker NEO.

In [5]:
!pip3 install gluoncv
!pip3 install xtarfile

     |████████████████████████████████| 1.3 MB 6.5 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_latest_p37/bin/python -m pip install --upgrade pip' command.
  Created wheel for xtarfile: filename=xtarfile-0.1.0-py3-none-any.whl size=3861 sha256=c9fed045788926646426605dd9be8e20900d093db12ecc19e5915f0d594d56e4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/14/ed/96/b06b7600d9d2eb6068fed8ded4282aeb3addf5b213dfe48819
Successfully built xtarfile
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_latest_p37/bin/python -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import mxnet as mx 
import gluoncv as gcv
import xtarfile as tarfile
import sagemaker
import boto3
import time

## Model download and hybridize
Use the following code to download the pre-trained model from MXNet model zoo. If you want to use models from other frameworks, you can use check out the [model preparation page](https://docs.aws.amazon.com/sagemaker/latest/dg/neo-compilation-preparing-model.html) on developer's guide to find out more information.

In the second line, we hybridize the model and run through it once with randomized input of the expected input size. For more information about hybridizing models, please checkout the [MXNet explanation on hybridize](https://mxnet.apache.org/versions/1.7.0/api/python/docs/tutorials/packages/gluon/blocks/hybridize.html).

In [2]:
net = gcv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
net.hybridize()
out = net(mx.nd.ones((1,3,224,224)))

88993KB [00:01, 70817.68KB/s]                           


[2021-11-22 05:02:06.045 ip-172-16-86-139:5515 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-22 05:02:06.157 ip-172-16-86-139:5515 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [3]:
net.export('yolo3_mobilenet1.0_coco')
tar = tarfile.open("model.tar.gz", "w:gz")
for name in ["yolo3_mobilenet1.0_coco-0000.params", "yolo3_mobilenet1.0_coco-symbol.json"]:
    tar.add(name)
tar.close()

## Tar the model, and upload our model to S3
Since in this notebook instance, there is already a execution role assiciated with it, we do not need to provision all the credentials. And in this example, for the simplicity, we are going to get the default bucket. SageMaker has a default bucket for each account (starts with sagemaker-) in the same region of this notebook instance. We are going to store all of the artifacts and compiled models within that default bucket. Feel free to change the bucket location if needed otherwise.

In [4]:
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
print(role)
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base("MXNet-yolo3-mobilenet10-Jetson-Nano")
prefix = compilation_job_name + "/model"
model_path = sess.upload_data(path="model.tar.gz", key_prefix=prefix)

data_shape = '{"data":[1,3,224,224]}'
target_platform = {'Os': 'LINUX', 'Arch': 'ARM64', 'Accelerator': 'NVIDIA'}
compiler_options = '{"trt-ver": "7.1.3", "cuda-ver": "10.2", "gpu-code": "sm_53"}'
framework = "MXNET"
framework_version = "1.7"
compiled_model_path = "s3://{}/{}/output".format(bucket, compilation_job_name)

arn:aws:iam::121984884871:role/smem-role


## Start compilation job
Start compilation job, and start a polling process to wait for the compilation job to succeed. It could take around 5 minutes.

In [5]:
# Create a SageMaker client so you can submit a compilation job
sagemaker_client = boto3.client('sagemaker')

response = sagemaker_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': model_path,
        'DataInputConfig': data_shape,
        'Framework': framework.upper()
    },
    OutputConfig={
        'S3OutputLocation': compiled_model_path,
        'TargetPlatform': target_platform,
        'CompilerOptions': compiler_options
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 900
    }
)
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Done!


## Repackage compiled model in zip file for Greengrass deployment
Since NEO compilation output is in the format of .tar.gz, but AWS IoT Greengrass only accepts .zip. We need the following step to convert the compiled model to convert the format in order for the archived file to be downloaded and unpacked by Greengrass service.

In [6]:
s3_client = boto3.client('s3')
neo_compiled_model = 'compiled-model.tar.gz'
s3_client.download_file(bucket, compilation_job_name+'/output/model-LINUX_ARM64_NVIDIA.tar.gz', neo_compiled_model)
!rm -rf model compiled-model.zip
!mkdir model
!tar zfxv compiled-model.tar.gz -C model/
!zip compiled-model.zip model/*
s3_client.upload_file('compiled-model.zip', bucket, '{}/model_zipped/model.zip'.format(compilation_job_name))

compiled.so
libdlr.so
compiled.params
dlr.h
compiled_model.json
compiled.meta
manifest
  adding: model/compiled.meta (deflated 77%)
  adding: model/compiled_model.json (deflated 89%)
  adding: model/compiled.params (deflated 78%)
  adding: model/compiled.so (deflated 8%)
  adding: model/dlr.h (deflated 83%)
  adding: model/libdlr.so (deflated 67%)
  adding: model/manifest (deflated 45%)


In [7]:
compiled_model_s3_url = f's3://{bucket}/{compilation_job_name}/model_zipped/model.zip'
compiled_model_s3_url


's3://sagemaker-us-west-2-121984884871/MXNet-yolo3-mobilenet10-Jetson-Nano-2021-11-22-05-02-16-204/model_zipped/model.zip'

# !!! PAUSE HERE !!!


# Section 2: Please only proceed if you have a shared library uploaded on S3 as instructed in the workshop guide.

### We need to pause here on this notebook and prepare the sample deepstream application before running the next steps.


In [8]:
packaged_deepstream_app_artifacts_S3_url = f's3://{bucket}/deepstream-people-counter/artifact/app.zip'

In [14]:
greengrass_recipe = {
  "RecipeFormatVersion": "2020-01-25",
  "ComponentName": "iot-greengrass-managed-deepstream-application",
  "ComponentVersion": "1.0.11",
  "ComponentDescription": "Component that runs CV program at the edge",
  "ComponentPublisher": "Amazon Web Services",
  "ComponentConfiguration": {
    "DefaultConfiguration": {
      "accessControl": {
        "aws.greengrass.ipc.pubsub": {
          "$component_name:pubsub:1": {
            "policyDescription": "Allows access to publish to all topics.",
            "operations": [
              "aws.greengrass#PublishToTopic"
            ],
            "resources": [
              "*"
            ]
          }
        }
      }
    }
  },
  "Manifests": [
    {
      "Platform": {
        "os": "linux"
      },
      "Lifecycle": {
        "Install": {
          "Script": "chmod 777 {artifacts:decompressedPath}/app/deepstream-test5-analytics ; rm -rf ~/.cache/gstreamer-1.0/registry.aarch64.bin"
        },
        "Run": {
          "Script": "sudo --preserve-env=COMPILED_MODEL_PATH {artifacts:decompressedPath}/app/deepstream-test5-analytics -c {artifacts:decompressedPath}/app/config/test5_config_file_src_infer_tlt_neo.txt"
        },
        "setenv": {
          "COMPILED_MODEL_PATH": "{artifacts:decompressedPath}/model/model"
        }
      },
      "Artifacts": [
        {
          "URI": compiled_model_s3_url,
          "Unarchive": "ZIP"
        },
        {
          "URI": packaged_deepstream_app_artifacts_S3_url,
          "Unarchive": "ZIP"
        }
      ]
    }
  ]
}

In [15]:
import json
gg_client = boto3.client('greengrassv2')
component_creation_ret = gg_client.create_component_version(inlineRecipe=json.dumps(greengrass_recipe).encode('utf-8'))

In [16]:
iot_client = boto3.client('iot')

thing_name = "jetsonDeepstreamDemo"
thing_group_name = "jetsonDeepstreamDemoGroup"
deployment_name = "deepstream-app-deployment"
response = gg_client.create_deployment(
    targetArn=iot_client.describe_thing(thingName = thing_name)['thingArn'],
    deploymentName=deployment_name,
    components={
        component_creation_ret['componentName']: {
            'componentVersion': component_creation_ret['componentVersion']
        }
    },
    deploymentPolicies={
        'failureHandlingPolicy': 'DO_NOTHING'
    }
)